# Segmentation trial acquisitions (calcium imaging)
* Segmentation of (motion corrected) images


In [69]:
import numpy as np
from cellpose import models
import napari
import os
from scripts.sample_db import SampleDB
from tifffile import imwrite, imread

# Load the sample database
db_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv'
sample_db = SampleDB()
sample_db.load(db_path)
print(sample_db)

# Loading experiment
sample_id = '20220426_RM0008_130hpf_fP1_f3'
exp = sample_db.get_sample(sample_id)
print(exp)

# Import model
model_path = r'D:\montruth\cellpose\models\CP_20230803_101131' 
model = models.CellposeModel(model_type=model_path, gpu=True)

# Making shortcuts of sample parameters/information
sample = exp.sample
raw_path = exp.paths["raw_path"]
anatomy_path = exp.paths["anatomy_path"]
em_path = exp.paths["em_path"]
n_planes = exp.params_lm.n_planes
n_frames = exp.params_lm.n_frames
n_slices = exp.params_lm.lm_stack_range
doubling = 2 if exp.params_lm.doubling else 1

# Getting paths of the trial acquisitions
trial_paths = os.listdir(os.path.join(raw_path, "trials"))

# Define the path for the preprocessed folder
preprocessed_folder = os.path.join(r"\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3", "preprocessed")
os.makedirs(preprocessed_folder, exist_ok=True)

# Define the path for the masks folder
masks_folder = os.path.join(preprocessed_folder, "..", "masks")
os.makedirs(masks_folder, exist_ok=True)

# Segment all trials per plane
all_images = []
all_masks = []
stack_to_mask = matched_stack
# Loop through each plane and process images
for plane in range(n_planes*doubling):
    plane_str = f"{plane:02d}"
    images =stack_to_mask[plane]
    #for trial_path in trial_paths:
    #    image_path = os.path.join(preprocessed_folder, f"transformed_plane_{plane:02d}_{trial_path}")
    #    images.append(imread(image_path).mean(axis=0))
    #images = np.array(images)
    #all_images.append(images)

    # Segment the images using Cellpose
    masks, flows, styles = model.eval(images, channels=[0, 0], cellprob_threshold=-6, flow_threshold=0, resample=False, augment=False, stitch_threshold=0.01)
    
    # Save the initial masks for each plane
    mask_plane_path = os.path.join(masks_folder, f"mask_plane_{plane_str}.tif")
    imwrite(mask_plane_path, masks)
    print(f"Mask for plane {plane} saved at {mask_plane_path}")

    # Store the masks for visualization
    all_masks.append(masks)

# Convert lists to numpy arrays for visualization
all_masks = np.array(all_masks)

SampleDB(sample_ids=['20220426_RM0008_130hpf_fP1_f3'])
sample=Sample(id='20220426_RM0008_130hpf_fP1_f3', parents_id='P1', genotype='aTubulin:GCamp6s x GAD1b:DsRed', phenotype='positive expression, nacre', dof='21.04.2022-09:30', hpf=130, body_length_mm=4) params_odor=ParamsOdor(odor_list=['Ala', 'Ser', 'Ctrl', 'TCA', 'GCA', 'TDCA', 'Cad', 'SA'], odor_sequence=['Ala', 'Ser', 'Ctrl', 'TCA', 'GCA', 'TDCA', 'Cad', 'SA', 'Cad', 'TDCA', 'GCA', 'TCA', 'Ser', 'Ctrl', 'Ala', 'SA', 'Ctrl', 'Cad', 'Ala', 'Ser', 'TDCA', 'GCA', 'TCA', 'SA'], odor_concentration_uM=[OdorConcentration(name='Ala', concentration_mM=100.0), OdorConcentration(name='Ser', concentration_mM=100.0), OdorConcentration(name='Ctrl', concentration_mM=100.0), OdorConcentration(name='TCA', concentration_mM=10.0), OdorConcentration(name='GCA', concentration_mM=10.0), OdorConcentration(name='TDCA', concentration_mM=10.0), OdorConcentration(name='Cad', concentration_mM=300.0), OdorConcentration(name='SA', concentration_mM=0.0)], n_tri

  0%|          | 0/23 [00:00<?, ?it/s]D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\cellpose\metrics.py:196: RuntimeWarning: invalid value encountered in divide
  iou = overlap / (n_pixels_pred + n_pixels_true - overlap)
100%|██████████| 23/23 [00:00<00:00, 129.20it/s]


Mask for plane 0 saved at \\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\preprocessed\..\masks\mask_plane_00.tif


100%|██████████| 23/23 [00:00<00:00, 120.41it/s]


Mask for plane 1 saved at \\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\preprocessed\..\masks\mask_plane_01.tif


...

Mask for plane 5 saved at \\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\preprocessed\..\masks\mask_plane_05.tif


100%|██████████| 23/23 [00:00<00:00, 116.16it/s]


Mask for plane 6 saved at \\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\preprocessed\..\masks\mask_plane_06.tif


100%|██████████| 23/23 [00:00<00:00, 164.27it/s]


Mask for plane 7 saved at \\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\preprocessed\..\masks\mask_plane_07.tif


In [68]:
matched_stack.shape

(8, 24, 256, 512)

In [71]:

# Visualize masks
viewer = napari.Viewer()
viewer.add_image(matched_stack, name='images_transformed')
viewer.add_labels(all_masks, name='masks')

D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\napari\plugins\_plugin_manager.py:555: UserWarning: Plugin 'napari-animated-gif-io' has already registered a function widget 'save as animated gif' which has now been overwritten
  warn(message=warn_message)


<Labels layer 'masks' at 0x1b2a11c8550>

In [17]:
all_masks.shape

(8, 24, 256, 512)

In [53]:
all_images_raw = []
for plane in range(n_planes*doubling):
    plane_str = f"{plane:02d}"
    images = []
    image_path = os.path.join(preprocessed_folder,f"sum_plane_{plane_str}_{trial_paths[0]}")
    print(imread(image_path).shape)
    images.append(imread(image_path))
    images = np.array(images)
    all_images_raw.append(images)

(24, 256, 512)
(24, 256, 512)
(24, 256, 512)
(24, 256, 512)
(24, 256, 512)
(24, 256, 512)
(24, 256, 512)
(24, 256, 512)


In [ ]:
all_images_rigid = []
for plane in range(n_planes*doubling):
    plane_str = f"{plane:02d}"
    images = []
    for trial_path in trial_paths:
        image_path = os.path.join(preprocessed_folder, "trials",  f"{trial_paths}")
        print(imread(image_path).shape)
        images.append(imread(image_path))
    images = np.array(images)
    all_images_raw.append(images)

In [44]:
np.array(all_images_rigid).shape

(8, 24, 24, 256, 512)

In [19]:
imwrite(r"\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\preprocessed\sum_transformed_trials.tif", np.array(all_images))


In [38]:
imwrite(r"\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\preprocessed\sum_rigid_corrected_trials.tif", np.array(all_images_rigid))

In [58]:
imwrite(r"\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\preprocessed\sum_raw_trials.tif", np.squeeze(all_images_raw))

In [29]:
np.array(all_images).shape

(8, 24, 256, 512)

In [43]:
np.array(all_images_rigid).shape

(8, 24, 24, 256, 512)

In [63]:
all_images_raw = imread(r"\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\preprocessed\sum_raw_trials.tif")
print(all_images_raw.shape)

all_images_rigid = imread(r"\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\preprocessed\sum_rigid_corrected_trials.tif")
print(all_images_rigid.shape)

all_images_transformed = imread(r"\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\preprocessed\sum_transformed_trials.tif")
print(all_images_transformed.shape)

(8, 24, 256, 512)
(8, 24, 256, 512)
(8, 24, 256, 512)


In [66]:
from skimage import exposure
viewer =napari.Viewer()
matched_stack = exposure.match_histograms(all_images_transformed,all_images_raw)
viewer.add_image(matched_stack, name='images_transformed')
viewer.add_image(all_images_rigid, name='images_rigid')
viewer.add_image(all_images_raw,  name='images_raw')

D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\napari\plugins\_plugin_manager.py:555: UserWarning: Plugin 'napari-animated-gif-io' has already registered a function widget 'save as animated gif' which has now been overwritten
  warn(message=warn_message)


<Image layer 'images_raw' at 0x1b2cb6590c0>

In [67]:
imwrite(r"\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\preprocessed\sum_transformed_trials.tif", matched_stack )